# Library Import #

In [1]:
import re
import nltk
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

# from keras.preprocessing import sequence
# from keras.utils import np_utils
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation, Lambda
# from keras.layers.embeddings import Embedding
# from keras.layers.recurrent import LSTM, SimpleRNN, GRU
# from keras.preprocessing.text import Tokenizer
# from collections import defaultdict
# from keras.layers.convolutional import Convolution1D
# from keras import backend as K
# import seaborn as sns
# import matplotlib.pyplot as plt
# from matplotlib import cm
# %matplotlib inline
# plt.style.use('ggplot')

Using TensorFlow backend.


# Data Import

we'll use all of the dates up to the end of 2014 as our training data and everything after as testing data.

In [3]:
data = pd.read_csv('../Data/Combined_News_DJIA.csv')
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']

# Data Preprocess

First, we transform the string of news into the  number of words as input.

## Text cleaning (this code didn't do much, I will add something this week)

### Training set

In [22]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

### Test set

In [ ]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

## Word Embedding (basic count, there are example for TFIDF in the following code)

### Training Set

In [17]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(1611, 31675)


### Test set

In [23]:
basictest = basicvectorizer.transform(testheadlines)
print(basictest.shape)

(378, 31675)


### Visualize the bag of word dictionary

In [20]:
print(basicvectorizer.get_feature_names())

['00', '000', '000bpd', '000ft', '000new', '000s', '000sq', '001', '004', '00am', '00pm', '011', '018', '02', '020', '0220', '0221', '03', '035', '037', '04', '045', '04am', '05', '053', '06', '060', '07', '07baku1268', '08', '080', '083', '08baku671', '09', '0900', '0930', '0935', '094', '0m', '10', '100', '1000', '10000', '1000s', '100bn', '100ds', '100ft', '100k', '100km', '100m', '100mil', '100s', '100th', '100x', '101', '101st', '102', '103', '1038', '104', '1044', '104m', '105', '105k', '106', '107', '107d', '108', '109', '10bars', '10bn', '10cm', '10gw', '10in', '10k', '10kg', '10km', '10m', '10million', '10s', '10th', '10x', '10yr', '10yrs', '11', '110', '110bn', '111', '112', '114', '1142', '115', '116', '116bn', '117', '119', '119th', '11b', '11bn', '11m', '11th', '11year', '11yo', '12', '120', '1200', '120bn', '121', '122', '122f', '123', '124', '125', '1255', '125mph', '126', '127', '1270', '128', '129', '12b', '12bn', '12k', '12m', '12th', '13', '130', '1300gmt', '130mph',

In [21]:
print(basictrain.toarray())

[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 2 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Model Training Example

### Logic Regression

Input: the counts of single words

### Build Logistic Regression Model

In [18]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Use test set to evaluate the performance of model

In [19]:
preds1 = basicmodel.predict(basictest)
acc1=accuracy_score(test['Label'], preds1)

In [14]:
print('Logic Regression 1 accuracy: ',acc1 )

Logic Regression 1 accuracy:  0.42592592592592593


The accuracy is only 0.42.

In [15]:
basicwords = basicvectorizer.get_feature_names()
basiccoeffs = basicmodel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
coeffdf.head(5)

,Word,Coefficient
19419,nigeria,0.497924
25261,self,0.452526
29286,tv,0.428011
15998,korea,0.425863
20135,olympics,0.425716


In [16]:
coeffdf.tail(5)

,Word,Coefficient
16949,low,-0.463116
3651,begin,-0.470454
25433,sex,-0.494555
24754,sanctions,-0.549725
24542,run,-0.587794


# something more complex, don't need to worry now

### Logic Regression 2

Algorithm: Logic Regression

Input: the counts of phrases with two connected words(exclude words which are too common like "a" ,"an" ,"the" and words too uncommon of which counts are too small )

We delete phrases of which frequency lower than 0.03 or higher than 0.97

In [10]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.97, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [11]:
print(advancedtrain.shape)

(1611, 657)


In [12]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

In [13]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds2 = advancedmodel.predict(advancedtest)
acc2=accuracy_score(test['Label'], preds2)

In [14]:
print('Logic Regression 2 accuracy: ', acc2)

Logic Regression 2 accuracy:  0.571428571429


The accuracy is higher than input of single words.

In [15]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Coefficient,Words
411,1.264377,set to
31,1.259669,and other
391,1.216202,right to
276,1.166895,likely to
14,1.128663,after the


In [16]:
advcoeffdf.tail(5)

,Coefficient,Words
131,-1.098193,fire on
40,-1.128193,around the
366,-1.144952,phone hacking
597,-1.146688,up in
452,-1.393351,the country


### Logic Regression 3

Algorithm: Logic Regression

Input: the counts of phrases with three connected words(exclude words which are too common like "a" ,"an" ,"the" and words too uncommon of which counts are too small )

In [17]:
advancedvectorizer = TfidfVectorizer( min_df=0.0039, max_df=0.1, max_features = 200000, ngram_range = (3, 3))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [18]:
print(advancedtrain.shape)

(1611, 2282)


In [19]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

In [20]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds3 = advancedmodel.predict(advancedtest)
acc3 = accuracy_score(test['Label'], preds3)

In [21]:
print('Logic Regression 3 accuracy: ', acc3)

Logic Regression 3 accuracy:  0.555555555556


The accuracy is lower than input of phrases with two connected words.

In [22]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Coefficient,Words
2075,1.076893,to the us
686,0.886762,in china the
1775,0.853562,the right to
1070,0.845685,nobel peace prize
448,0.832537,first time since


In [23]:
advcoeffdf.tail(5)

,Coefficient,Words
1425,-0.912953,said to be
838,-0.933243,in west africa
1317,-0.943155,phone hacking scandal
152,-0.972078,around the world
779,-1.154834,in the country


## Naive Bayes

### NBayes 1

In [24]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.7, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [25]:
print(advancedtrain.shape)

(1611, 529)


In [26]:
advancedmodel = MultinomialNB(alpha=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds4 = advancedmodel.predict(advancedtest)
acc4=accuracy_score(test['Label'], preds4)

In [27]:
print('NBayes 1 accuracy: ', acc4)

NBayes 1 accuracy:  0.513227513228


In [28]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Coefficient,Words
234,-5.180422,israel
497,-5.252936,was
94,-5.312797,china
318,-5.317049,not
394,-5.329156,says


In [29]:
advcoeffdf.tail(5)

,Coefficient,Words
204,-6.948273,held
446,-6.951188,tells
426,-6.952040,story
290,-6.970522,monday
268,-6.970642,lost


### NBayes 2

In [30]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [31]:
print(advancedtrain.shape)

(1611, 631)


In [32]:
advancedmodel = MultinomialNB(alpha=0.0001)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds5 = advancedmodel.predict(advancedtest)
acc5 = accuracy_score(test['Label'], preds5)

In [33]:
print('NBayes 2 accuracy: ', acc5)

NBayes 2 accuracy:  0.542328042328


In [34]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Coefficient,Words
49,-5.495760,at least
185,-5.499764,in china
247,-5.543152,it is
259,-5.556370,killed in
487,-5.594222,the united


In [35]:
advcoeffdf.tail(5)

,Coefficient,Words
386,-7.176794,say the
65,-7.190919,been arrested
172,-7.212155,if it
84,-7.218639,children in
383,-7.323029,said the


## Random Forest

### RF 1

In [36]:
advancedvectorizer = TfidfVectorizer( min_df=0.01, max_df=0.99, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [37]:
print(advancedtrain.shape)

(1611, 4565)


In [38]:
advancedmodel = RandomForestClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds6 = advancedmodel.predict(advancedtest)
acc6 = accuracy_score(test['Label'], preds6)

In [39]:
print('RF 1 accuracy: ', acc6)

RF 1 accuracy:  0.52380952381


### RF 2

In [40]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [41]:
print(advancedtrain.shape)

(1611, 631)


In [42]:
advancedmodel = RandomForestClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds7 = advancedmodel.predict(advancedtest)
acc7 = accuracy_score(test['Label'], preds7)

In [43]:
print('RF 2 accuracy: ', acc7)

RF 2 accuracy:  0.539682539683


## Gradient Boosting Machines

### GBM 1

In [44]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.9, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [45]:
print(advancedtrain.shape)

(1611, 541)


In [46]:
advancedmodel = GradientBoostingClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds8 = advancedmodel.predict(advancedtest.toarray())
acc8 = accuracy_score(test['Label'], preds8)

In [47]:
print('GBM 1 accuracy: ', acc8)

GBM 1 accuracy:  0.473544973545


### GBM 2

In [48]:
advancedvectorizer = TfidfVectorizer( min_df=0.02, max_df=0.175, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [49]:
print(advancedtrain.shape)

(1611, 1169)


In [50]:
advancedmodel = GradientBoostingClassifier()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds9 = advancedmodel.predict(advancedtest.toarray())
acc9 = accuracy_score(test['Label'], preds9)

In [51]:
print('GBM 2 accuracy: ', acc9)

GBM 2 accuracy:  0.547619047619


## Stochastic Gradient Descent Classifier

### SGDClassifier 1

In [52]:
advancedvectorizer = TfidfVectorizer( min_df=0.2, max_df=0.8, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [53]:
print(advancedtrain.shape)

(1611, 202)


In [54]:
advancedmodel = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds10 = advancedmodel.predict(advancedtest.toarray())
acc10 = accuracy_score(test['Label'], preds10)

In [55]:
print('SGDClassifier 1: ', acc10)

SGDClassifier 1:  0.505291005291


### SGDClassifier 2

In [56]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [57]:
print(advancedtrain.shape)

(1611, 631)


In [58]:
advancedmodel = SGDClassifier(loss='modified_huber', n_iter=5, random_state=0, shuffle=True)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds11 = advancedmodel.predict(advancedtest.toarray())
acc11 = accuracy_score(test['Label'], preds11)

In [59]:
print('SGDClassifier 2: ', acc11)

SGDClassifier 2:  0.542328042328


## Naive Bayes SVM

In [60]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):

    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

        #p_c /= np.linalg.norm(p_c, ord=1)
        #ratios[c] = np.log(p_c / (1 - p_c))


def f1_class(pred, truth, class_val):
    n = len(truth)

    truth_class = 0
    pred_class = 0
    tp = 0

    for ii in range(0, n):
        if truth[ii] == class_val:
            truth_class += 1
            if truth[ii] == pred[ii]:
                tp += 1
                pred_class += 1
                continue;
        if pred[ii] == class_val:
            pred_class += 1

    precision = tp / float(pred_class)
    recall = tp / float(truth_class)

    return (2.0 * precision * recall) / (precision + recall)


def semeval_senti_f1(pred, truth, pos=2, neg=0): 

    f1_pos = f1_class(pred, truth, pos)
    f1_neg = f1_class(pred, truth, neg)

    return (f1_pos + f1_neg) / 2.0;


def main(train_file, test_file, ngram=(1, 3)):
    print('loading...')
    train = pd.read_csv(train_file, delimiter='\t', encoding='utf-8', header=0,
                        names=['text', 'label'])

    # to shuffle:
    #train.iloc[np.random.permutation(len(df))]

    test = pd.read_csv(test_file, delimiter='\t', encoding='utf-8', header=0,
                        names=['text', 'label'])

    print('vectorizing...')
    vect = CountVectorizer()
    classifier = NBSVM()

    # create pipeline
    clf = Pipeline([('vect', vect), ('nbsvm', classifier)])
    params = {
        'vect__token_pattern': r"\S+",
        'vect__ngram_range': ngram, 
        'vect__binary': True
    }
    clf.set_params(**params)

    #X_train = vect.fit_transform(train['text'])
    #X_test = vect.transform(test['text'])

    print('fitting...')
    clf.fit(train['text'], train['label'])

    print('classifying...')
    pred = clf.predict(test['text'])
   
    print('testing...')
    acc = accuracy_score(test['label'], pred)
    f1 = semeval_senti_f1(pred, test['label'])
    print('NBSVM: acc=%f, f1=%f' % (acc, f1))

### NBSVM 1

In [61]:
advancedvectorizer = TfidfVectorizer( min_df=0.1, max_df=0.8, max_features = 200000, ngram_range = (1, 1))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1611, 535)


In [62]:
advancedmodel = NBSVM(C=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds12 = advancedmodel.predict(advancedtest)
acc12 = accuracy_score(test['Label'], preds12)

In [63]:
print('NBSVM 1: ', acc12)

NBSVM 1:  0.478835978836


### NBSVM 2

In [64]:
advancedvectorizer = TfidfVectorizer( min_df=0.031, max_df=0.2, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
print(advancedtrain.shape)

(1611, 613)


In [65]:
advancedmodel = NBSVM(C=0.01)
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds13 = advancedmodel.predict(advancedtest)
acc13 = accuracy_score(test['Label'], preds13)

In [66]:
print('NBSVM 2: ', acc13)

NBSVM 2:  0.595238095238


## Deep Learning

### MLP

In [67]:
batch_size = 32
nb_classes = 2
advancedvectorizer = TfidfVectorizer( min_df=0.04, max_df=0.3, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
print(advancedtrain.shape)

(1611, 401)


In [68]:
X_train = advancedtrain.toarray()
X_test = advancedtest.toarray()

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(train["Label"])
y_test = np.array(test["Label"])

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


# pre-processing: divide by max and substract mean
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.mean(X_train)
X_train -= mean
X_test -= mean

input_dim = X_train.shape[1]

# Here's a Deep Dumb MLP (DDMLP)
model = Sequential()
model.add(Dense(256, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

print("Training...")
model.fit(X_train, Y_train, nb_epoch=2, batch_size=16, validation_split=0.15, show_accuracy=True)

print("Generating test predictions...")
preds14 = model.predict_classes(X_test, verbose=0)
acc14 = accuracy_score(test["Label"], preds14)

X_train shape: (1611, 401)
X_test shape: (378, 401)
Training...


/opt/conda/lib/python3.6/site-packages/Keras-1.2.2-py3.6.egg/keras/models.py:654: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Train on 1369 samples, validate on 242 samples
Epoch 1/2
1369/1369 [==============================] - 0s - loss: 0.6953 - val_loss: 0.6879
Epoch 2/2
1369/1369 [==============================] - 0s - loss: 0.6685 - val_loss: 0.6850
Generating test predictions...


In [69]:
print('prediction accuracy: ', acc14)

prediction accuracy:  0.547619047619


### LSTM

In [70]:
max_features = 10000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1
maxlen = 200
batch_size = 32
nb_classes = 2

In [71]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=max_features)
tokenizer.fit_on_texts(trainheadlines)
sequences_train = tokenizer.texts_to_sequences(trainheadlines)
sequences_test = tokenizer.texts_to_sequences(testheadlines)

In [72]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Pad sequences (samples x time)
X_train shape: (1611, 200)
X_test shape: (378, 200)


In [73]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2)) 
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=3,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
preds15 = model.predict_classes(X_test, verbose=0)
acc15 = accuracy_score(test['Label'], preds15)

Build model...
Train...


/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1611 samples, validate on 378 samples
Epoch 1/3
1611/1611 [==============================] - 50s - loss: 0.6942 - acc: 0.5208 - val_loss: 0.6935 - val_acc: 0.5079
Epoch 2/3
1611/1611 [==============================] - 51s - loss: 0.6722 - acc: 0.5971 - val_loss: 0.6904 - val_acc: 0.5106
Epoch 3/3
378/378 [==============================] - 3s     
Test score: 0.683092702633
Test accuracy: 0.566137566138
Generating test predictions...


In [74]:
print('prediction accuracy: ', acc15)

prediction accuracy:  0.566137566138


## CNN

In [75]:
nb_filter = 120
filter_length = 2
hidden_dims = 120
nb_epoch = 2

In [76]:
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))

def max_1d(X):
    return K.max(X, axis=1)

model.add(Lambda(max_1d, output_shape=(nb_filter,)))
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [77]:
print('Train...')
model.fit(X_train, Y_train, batch_size=32, nb_epoch=1,
          validation_data=(X_test, Y_test))
score, acc = model.evaluate(X_test, Y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)


print("Generating test predictions...")
preds16 = model.predict_classes(X_test, verbose=0)
acc16 = accuracy_score(test['Label'], preds16)

Train...


/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1611 samples, validate on 378 samples
Epoch 1/1
352/378 [==========================>...] - ETA: 0sTest score: 0.69742725829
Test accuracy: 0.507936509829
Generating test predictions...


In [78]:
print('prediction accuracy: ', acc16)

prediction accuracy:  0.507936507937
